In [25]:
!pip install gym

In [32]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 311kB/s eta 0:00:011   |▋                               | 1.7MB 1.1MB/s eta 0:01:23     |▋                               | 1.8MB 1.1MB/s eta 0:01:23     |█▏                              | 3.4MB 1.1MB/s eta 0:01:21     |██▏                             | 6.2MB 7.4MB/s eta 0:00:12     |█████▎                          | 15.4MB 9.6MB/s eta 0:00:09     |█████▊                          | 16.6MB 9.6MB/s eta 0:00:08     |███████▎                        | 20.9MB 8.2MB/s eta 0:00:09     |███████████                     | 31.7MB 3.3MB/s eta 0:00:19     |███████████                     | 31.8MB 3.3MB/s eta 0:00:19     |███████████▍                    | 33.1MB 3.3MB/s eta 0:00:19     |████████████▋                   | 36.4MB 3.3MB/s eta 0:00:18     |█████████████▍                  | 38.6MB 16.2MB/s eta 0:00:04     |█████████████▌                  | 39.0MB 16.2MB/s eta 0:00:04     |██████████████▉                 | 43.1MB 16.2MB/s eta 0:00:04     |██████

In [35]:
import pandas as pd
import numpy as np
import gym
from stable_baselines import A2C,DQN
# from stable_baselines.common.policies import MlpPolicy
# from stable_baselines.common.vec_env import DummyVecEnv

# from dmarket.environments import SingleAgentTrainingEnv
# from dmarket.agents import UniformRandomAgent, GymRLAgent
# from dmarket.info_settings import OfferInformationSetting
# from dmarket.engine import MarketEngine

KeyError: "Registering two gradient with name 'LSTMBlockCell'! (Previous registration was in register /opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/registry.py:66)"

Let's create a training environment for baselines with 5 fixed buyers and sellers and 1 RL agent to be trained

In [ ]:
fixed_agents = [
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('seller', 90),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
    UniformRandomAgent('buyer', 110),
]

rl_agent = GymRLAgent('buyer', 110, discretization=20)
setting = OfferInformationSetting(5)

def get_env(rl_agent, fixed_agents, setting):
    return SingleAgentTrainingEnv(rl_agent, fixed_agents, setting)

env = DummyVecEnv([lambda: get_env(rl_agent, fixed_agents, setting)]) # wrap it for baselines

Initialize a deep-Q RL model with MLP policy and high learning rate:

In [ ]:
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.05)

In [ ]:
%%time
model.learn(total_timesteps=10000)

Let's put this agent to the test

In [ ]:
rl_agent.model = model

In [ ]:
def get_reward(agent, deals):
    if not agent.name in deals:
        return 0

    deal_price = deals[agent.name]
    if agent.role == 'buyer':
        return agent.reservation_price - deal_price
    else:
        return deal_price - agent.reservation_price

def play_games(agents, setting, n_games=100, max_steps=30):
    buyer_ids =  [
        agent.name
        for agent in agents
        if agent.role == 'buyer'
    ]
    seller_ids =  [
        agent.name
        for agent in agents
        if agent.role == 'seller'
    ]
    ids = set(buyer_ids + seller_ids)
    market = MarketEngine(buyer_ids, seller_ids, max_steps=max_steps)
    
    rewards = pd.DataFrame(0, index=np.arange(n_games), columns=ids)
    for game_idx in range(n_games):
        while market.done != ids:
            observations = setting.get_states(ids, market)
            unmatched_agents = [
                agent for agent in agents
                if agent.name not in market.done
            ]
            offers = {
                agent.name: agent.get_offer(observations[agent.name])
                for agent in unmatched_agents
            }
            deals = market.step(offers)
            for agent in unmatched_agents:
                rewards[agent.name][game_idx] = get_reward(agent, deals)
        market.reset()
    return rewards

In this simple situation, the agent can get decent rewards:

In [ ]:
play_games(fixed_agents + [rl_agent], setting, 10)

In [ ]:
play_games(fixed_agents + [rl_agent], setting, 100).describe()

Let's see if it can act as a seller as well:

In [ ]:
rl_seller = GymRLAgent('seller', 90, model=model, name='myseller')

In [ ]:
play_games(fixed_agents + [rl_seller], setting, 100).describe()

We see it performs slightly worse.